In [105]:
import os
os.path.abspath("")

'/home/yuxiangliao/PhD'

In [106]:
import os, sys
from loguru import logger

LOG_ROOT = os.path.abspath("./")
LOG_FILE = LOG_ROOT + "/logs/sr-3.log"

# Remove all handlers and reset stderr
logger.remove(handler_id=None)
logger.add(
    LOG_FILE,
    level="TRACE",
    mode="w",
    backtrace=False,
    diagnose=True,
    colorize=False,
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
)
logger.info("\r\n" + ">" * 29 + "\r\n" + ">>> New execution started >>>" + "\r\n" + ">" * 29)
# To filter log level: TRACE=5, DEBUG=10, INFO=20, SUCCESS=25, WARNING=30, ERROR=40, CRITICAL=50
logger.add(sys.stdout, level="INFO", filter=lambda record: record["level"].no < 40, colorize=True)
logger.add(sys.stderr, level="ERROR", backtrace=False, diagnose=True, colorize=True)

9

# Install Metamap

Follow the following instructions:
- Install Metamap2020: https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/documentation/Installation.html 
- Install additional datasets (2022 Specialist Lexicon, 2022AA UMLS NLM Datasets): https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/additional-tools/DataSetDownload.html

# Processing

## Check if the servers started
- taggerServer
- DisambiguatorServer

In [107]:
import os
cmd = 'ps -ef | grep java'
out = os.popen(cmd)
print(out.read())

yuxiang+   31170    1377  0 Jul08 ?        00:06:44 java -Dtaggerserver.port=1795 -DlexFile=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/data/lexDB.serial -DngramOne=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/data/ngramOne.serial -cp /home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/Tagger_server/lib/taggerServer.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/MedPost-SKR/Tagger_server/lib/mps.jar taggerServer
yuxiang+   31218    1377  0 Jul08 ?        00:07:20 java -Xmx2g -Dserver.config.file=/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/config/disambServer.cfg -classpath /home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/metamapwsd.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/utils.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/lucene-core-3.0.1.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/monq-1.1.1.jar:/home/yuxiangliao/PhD/UMLS/Metamap/public_mm/WSD_Server/lib/wsd.jar:/home/

## Check metamap human readable output

In [108]:
# import subprocess, shlex
# text =  "There is no focal consolidation, pleural effusion or pneumothorax.  Cardiomediastinal silhouette and hilar contours are otherwise unremarkable."
# input_command = f"echo -e {text}"
# input_process = subprocess.Popen(shlex.split(input_command), stdout=subprocess.PIPE)
# meta_command = "metamap -V NLM -Z 2022AA -A --silent -I"
# metamap_process = subprocess.Popen(shlex.split(meta_command), stdout=subprocess.PIPE, stdin=input_process.stdout)
# output, error = metamap_process.communicate()
# print(output.decode())

## Load data

In [109]:
import pandas as pd
REPORT_PATH = "/home/yuxiangliao/PhD/data/mimic_cxr_reports_core.json"
df = pd.read_json(REPORT_PATH,orient="records",lines=True)
print(df)

id_list = df.loc[:,'sid'].to_list()
findings_list = df.loc[:,'findings'].to_list()
impression_list = df.loc[:,'impression'].to_list()
pfi_list = df.loc[:,'provisional_findings_impression'].to_list()
fai_list = df.loc[:,'findings_and_impression'].to_list()

DATA_SIZE = len(id_list)

              pid        sid  \
0       p10000032  s50414267   
1       p10000032  s53189527   
2       p10000032  s53911762   
3       p10000032  s56699142   
4       p10000764  s57375967   
...           ...        ...   
227830  p19999442  s58708861   
227831  p19999733  s57132437   
227832  p19999987  s55368167   
227833  p19999987  s58621812   
227834  p19999987  s58971208   

                                                 findings  \
0       There is no focal consolidation, pleural effus...   
1       The cardiac, mediastinal and hilar contours ar...   
2       Single frontal view of the chest provided. \n ...   
3       The lungs are clear of focal consolidation, pl...   
4       PA and lateral views of the chest provided.   ...   
...                                                   ...   
227830  ET tube ends 4.7 cm above the carina.  NG tube...   
227831  The lungs are clear, and the cardiomediastinal...   
227832  There has been interval extubation and improve...   
22783

## Run multiprocessing in jupyter

Construct metama command

In [110]:
import subprocess, shlex
# Documentation: https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/MM_2016_Usage.pdf
def get_metamap_command():
    command = format_command_arg("metamap")
    command += format_command_arg("-V NLM")                # Data Version: -V (--mm data version) [Base, USAbase, NLM]
    command += format_command_arg("-Z 2022AA")             # Knowledge Source: -Z (--mm data year)
    command += format_command_arg("-A")                    # Data Model: [-A (--strict model), -C (--relaxed model)]
    command += format_command_arg("--silent")              # Hide Header Output: --silent
    command += format_command_arg("--JSONn")               # Output format: [-q (--machine output), --XMLf, --XMLn, --XMLf1, --XMLn1, --JSONf, --JSONn, -N (--fielded mmi output), -F (--formal tagger output)]
    # command += " --conj"                                   # Turn on Conjunction Processing
    # command += " -y"                                       # Word-Sense Disambiguation: -y (--word sense disambiguation)
    # UDA_path = "/home/yuxiangliao/PhD/UMLS/custom-resources/custom-word-replacement"
    # command += format_command_arg(f"--UDA {UDA_path}")     # User-Defined Acronyms/Abbreviations (word replacement): --UDA <file>
    # semantic_types = "virs,cgab,acab,ffas,bpoc,medd,tmco,qlco,qnco,bsoj,blor,fndg,sosy,topp,ortf,patf,dsyn,inpo"
    # commend += f"-J {semantic_types}"                      # Retain only Concepts with Specified Semantic Types: -J (--restrict to sts) <list>
    # command += format_command_arg("-I")                    # For human readable output
    return command

def format_command_arg(arg):
    return " " + arg

def run_metamap(startIndex,batch_size):
    endIndex = startIndex + batch_size if startIndex + batch_size < DATA_SIZE else DATA_SIZE
    input_list = [(record if record else "None") for record in findings_list[startIndex:endIndex]]
    input = repr("\n\n".join(input_list))
    input_command = f"echo -e {input}"
    input_process = subprocess.Popen(shlex.split(input_command), stdout=subprocess.PIPE)
    
    meta_command = get_metamap_command()
    metamap_process = subprocess.Popen(shlex.split(meta_command), stdout=subprocess.PIPE, stdin=input_process.stdout)
   
    output_bytes, error_bytes = metamap_process.communicate()
    if error_bytes:
        logger.error(error_bytes.decode())
    return output_bytes.decode(), [startIndex,endIndex]

Data Object for JSON output

In [111]:
class Concept(object):
    def __init__(self, sourceTokens:list, startPosList:list, lengthList:list, umlsCUI:str, preferedName:str, hitTerm:str, categories:list, isHead:int, isNegated:int):
        self.sourceTokens = sourceTokens
        self.startPosList = startPosList
        self.lengthList = lengthList
        self.umlsCUI = umlsCUI
        self.preferedName = preferedName
        self.hitTerm = hitTerm
        self.categories = categories
        self.isHead =  isHead
        self.isNegated =  isNegated
        
class ConceptGroup(object):
    def __init__(self):
        self.concepts = []
    def addConcept(self, concept:Concept):
        self.concepts.append(concept)
        
class SyntaxChunk(object):
    def __init__(self, text:str, lexicalMatch:str, syntaxType:str, partOfSpeech:str, tokens:list):
        self.text = text # The original form of the text (case sensitive)
        self.lexicalMatch = lexicalMatch
        self.syntaxType = syntaxType
        self.partOfSpeech = partOfSpeech
        self.tokens = tokens
                
class Phrase(object):
    def __init__(self, text:str, startPos:int, length:int):
        self.text = text
        self.startPos = startPos
        self.length = length
        self.syntaxChunks = []
        self.mappings = []
    def addSyntaxChunk(self, syntaxChunk:SyntaxChunk):
        self.syntaxChunks.append(syntaxChunk)
    def addConceptGroup(self, conceptGroup:ConceptGroup):
        self.mappings.append(conceptGroup)
        
        
class Sentence(object):
    def __init__(self, text:str, startPos:int, length:int):
        self.text = text
        self.startPos = startPos
        self.length = length
        self.phrases = []
    def addPhrase(self, phrase:Phrase):
        self.phrases.append(phrase)

class Negation(object):
    def __init__(self, text:str, triStartPosList:list, triLengthList:list, conceptsCUIs:list, tarStartPosList:list, tarLengthList:list):
        self.trgger = {
            'text': text,
            'startPosList': triStartPosList,
            'lengthList': triLengthList,
        }
        self.tarrget = {
            'conceptsCUIs': conceptsCUIs,
            'startPosList': tarStartPosList,
            'lengthList': tarLengthList,
        }

class Section(object):
    def __init__(self, name:str):
        self.name = name
        self.text = "" # context
        self.sentences = []
        self.negations = []
    def addSentence(self, sentence:Sentence):
        self.sentences.append(sentence)
        self.text += sentence.text
    def addNegation(self, negation:Negation):
        self.negations.append(negation)
        
class Record(object):
    def __init__(self, sid:str):
        self.sid = sid
        self.sections = []
    def addSection(self, section:Section):
        self.sections.append(section)
    def getFindingSection(self) -> Section:
        assert self.sections[0].name == "findings"
        return self.sections[0]
        
class Records(object):
    def __init__(self):
        self.records = []
    def addRecord(self, record:Record):
        self.records.append(record)

Methods to resolve specific JSON subtags

In [112]:
def resolveSyntaxUnit(syntaxUnit):
    text = syntaxUnit['InputMatch']
    syntaxType = syntaxUnit['SyntaxType']
    tokens = syntaxUnit['Tokens']
    # Add punc to token list
    if not tokens:
        logger.trace(f"Empty token detected: SyntaxType:{syntaxType}, InputMatch:{text}")
        tokens = [text]
    try:
        lexicalMatch = syntaxUnit['LexMatch']
        partOfSpeech = syntaxUnit['LexCat']
    except KeyError:
        lexicalMatch = ""
        partOfSpeech = ""
    if text.lower() != lexicalMatch and text.isalnum():
        logger.trace(f"text:[{text}], lexicalMatch:[{lexicalMatch}]")
    return SyntaxChunk(text, lexicalMatch, syntaxType, partOfSpeech, tokens)

def resolveConcept(mappingCandidate):
    sourceTokens = mappingCandidate['MatchedWords']
    startPosList = [int(i['StartPos']) for i in mappingCandidate['ConceptPIs']]
    lengthList = [int(i['Length']) for i in mappingCandidate['ConceptPIs']]
    umlsCUI = mappingCandidate['CandidateCUI']
    preferedName = mappingCandidate['CandidatePreferred']
    hitTerm = mappingCandidate['CandidateMatched']
    categories = mappingCandidate['SemTypes']
    isHead = 1 if mappingCandidate['IsHead'] == "yes" else 0
    isNegated = 1 if mappingCandidate['Negated'] == "1" else 0
    return Concept(sourceTokens, startPosList, lengthList, umlsCUI, preferedName, hitTerm, categories, isHead, isNegated)

def resolveNegation(negation):
    trigger = negation['NegTrigger']
    triggerStartPosList = [int(i['StartPos']) for i in negation['NegTriggerPIs']]
    triggerLengthList = [int(i['Length']) for i in negation['NegTriggerPIs']]
    conceptCUIs = [i['NegConcCUI'] for i in negation['NegConcepts']]
    targetStartPosList = [int(i['StartPos']) for i in negation['NegConcPIs']]
    targetLengthList = [int(i['Length']) for i in negation['NegConcPIs']]
    return Negation(trigger, triggerStartPosList, triggerLengthList, conceptCUIs, targetStartPosList, targetLengthList)
    

Method to resolve JSON format output

In [113]:
def parseMetamapJSON(json_obj,id_subList) -> Records:
    records = Records()
    for _idx, _document in enumerate(json_obj['AllDocuments']):
        # print(_document.keys())
        # print(record['Document']['Negations'])
        record = Record(id_subList[_idx])
        section = Section("findings")
        for _utterance in _document['Document']['Utterances']:
            # print(_utterance.keys())
            sentence = Sentence(text=_utterance['UttText'], startPos=int(_utterance['UttStartPos']), length=int(_utterance['UttLength']))
            for _phrase in _utterance['Phrases']:
                # print(_phrase.keys())
                phrase = Phrase(text=_phrase['PhraseText'], startPos=int(_phrase['PhraseStartPos']), length=int(_phrase['PhraseLength']))
                for _syntaxUnit in _phrase['SyntaxUnits']:
                    # print(_syntaxUnit.keys())
                    syntaxChunk = resolveSyntaxUnit(_syntaxUnit)
                    phrase.addSyntaxChunk(syntaxChunk)
                for _mapping in _phrase['Mappings']:
                    # print(_mapping.keys())
                    conceptGroup = ConceptGroup()
                    for _mappingCandidate in _mapping['MappingCandidates']:
                        # print(_mappingCandidate.keys())
                        concept = resolveConcept(_mappingCandidate)
                        conceptGroup.addConcept(concept)
                    phrase.addConceptGroup(conceptGroup)
                sentence.addPhrase(phrase)
            section.addSentence(sentence)
        for _negation in _document['Document']['Negations']:
            negation = resolveNegation(_negation)
            section.addNegation(negation)
        record.addSection(section)
        records.addRecord(record)
    return records

Methods to align the metamap output to the spacy output

In [114]:
from spacy.tokens import Doc
from operator import itemgetter

def align(baseList:Doc, inputTokenGroups):
    alignment= [-1] * len(baseList)
    for id, tokenGroup in enumerate(inputTokenGroups):
        alignment[tokenGroup.start:tokenGroup.end] = [id] * (tokenGroup.end-tokenGroup.start)
    return alignment

def align_byIndex(baseList:Doc, inputIndexGroups):
    alignment= [-1] * len(baseList)
    for id, indexGroup in enumerate(inputIndexGroups):
        alignment[indexGroup[0]:indexGroup[-1]+1] = [id] * len(indexGroup)
    return alignment

def align_byIndex_individually_withData_noOverlap(baseList:Doc, inputIndexGroups_withData):
    alignment= [-1] * len(baseList)
    for id, indexGroup_withData in enumerate(inputIndexGroups_withData):
        indexGroup = indexGroup_withData['indices']
        extra_str = indexGroup_withData['extra_str']
        for index in indexGroup:
            alignment[index] = f"{id}|{extra_str}"
    return alignment

def align_byIndex_individually_withData(baseList:Doc, inputIndexGroups_withData):
    alignment= [-1] * len(baseList)
    for id, indexGroup_withData in enumerate(inputIndexGroups_withData):
        indexGroup = indexGroup_withData['indices']
        extra_str = indexGroup_withData['extra_str']
        for index in indexGroup:
            if alignment[index] == -1:
                alignment[index] = [extra_str]
            else:
                alignment[index].append(extra_str) 
    return alignment

def getTokenOffset(baseText:str, inputTokens):
    startPos = 0
    offset= []
    for token in inputTokens:
        offsetPos = baseText.find(token.text, startPos, len(baseText))
        offset.append(offsetPos)
        startPos = offsetPos + len(token.text)
    return offset

def resolveTokenIndices_byPosition(tokenOffset, startPos, length) -> list:
    indicesList = []
    doInsert = False
    posPointer = startPos
    for i, currPos in enumerate(tokenOffset):
        nextPos = tokenOffset[i+1] if i + 1 < len(tokenOffset) else tokenOffset[i] + 99
        if not doInsert and posPointer >= currPos and posPointer < nextPos:
            doInsert = True
            posPointer = startPos + length - 1
        elif doInsert and posPointer < currPos: 
            break # break the loop in advance, othewise will stop when finish the loop.
        if doInsert:
            indicesList.append(i)
    return indicesList

def resolveTokenIndices_byPosition_multiToken(tokenOffset, startPosList, lengthList) -> list:
    idxList_3d = [resolveTokenIndices_byPosition(tokenOffset, startPos, length) for startPos, length in zip(startPosList, lengthList)]
    idxList_flatten = [idx for idxList in idxList_3d for idx in idxList]
    return idxList_flatten

def trimIndices(_indices, keepNum):
    interval = []
    for id, current in enumerate(_indices):
        if id == len(_indices)-1:
            break
        nextid = id+1
        next = _indices[nextid]
        interval.append(next-current)
    interval_withIdx = list(enumerate(interval))
    trimed_list = sorted(interval_withIdx,key=itemgetter(1))[0:keepNum-1]
    idx_remained = set()
    for i in trimed_list:
        idx_remained.add(i[0])
        idx_remained.add(i[0]+1)
    return [_indices[i] for i in idx_remained]

def replPunc(matchObj):
    if matchObj.string == matchObj.group(0):
        return matchObj.string
    else:
        return ""

In [115]:
import jsonpickle

def classToJSON(obj) -> str:
    return jsonpickle.encode(obj,unpicklable=False)

Format metamap outputs so that it can be aligned to spacy tokens

In [157]:
def formatMetamapRecord(metamapRecord):
    phrases = [phrase for sentence in metamapRecord.getFindingSection().sentences for phrase in sentence.phrases]
    tokenOffset = spacyOutput.loc[:,SPACY_COLUMN_NAME['token_offset']].tolist()
    phraseIdxGroups = []
    syntaxChunkIdxGroups_withData = []
    conceptIdxGroup_withData = []
    negTriggerGroups_withData = []
    negTargetGroups_withData = []
    conceptGroupId = 0
    negationGroupId = 0
    for phrase in phrases:
        phraseIdxList = resolveTokenIndices_byPosition(tokenOffset, phrase.startPos, phrase.length)
        phraseIdxGroups.append(phraseIdxList)
        tokenSubset = spacyOutput.loc[phraseIdxList[0]:phraseIdxList[-1],SPACY_COLUMN_NAME['token']]
        syntaxTokenIdxOffset = 0
        print(f"tokenSubset:{tokenSubset.tolist()}")
        for syntaxChunk in phrase.syntaxChunks:
            print(f"syntaxChunk.text:{syntaxChunk.text}")
            print(f"syntaxChunk.tokens:{syntaxChunk.tokens}")
            syntaxTokenIdxOffset_next = syntaxTokenIdxOffset+len(syntaxChunk.tokens)
            # Because Metamap does not provide token offset position for its syntax chunk (unit).
            # Hence, we designed a three-steps check to avoid wrong recognization of a token that appear multiple times across 
            # different syntaxChunk within the current phrase.
            # 1. We split the phrase token list into syntaxChunk token list for aligement.
            tokenSubset_chunk = tokenSubset.iloc[syntaxTokenIdxOffset:syntaxTokenIdxOffset_next]
            syntaxChunkIndices = tokenSubset_chunk[tokenSubset_chunk.str.lower().isin(syntaxChunk.tokens)].index.tolist()
            print(f"syntaxChunkIndices-1:{syntaxChunkIndices}")
            if len(syntaxChunkIndices) != len(syntaxChunk.tokens):
                # 2. If it doesn't work, then we remove the symbol marks in tokens and try again.
                syntaxChunkIndices = tokenSubset_chunk[tokenSubset_chunk.str.replace(r'[\W]+', replPunc, regex=True).str.lower().isin(syntaxChunk.tokens)].index.tolist()
                print(f"syntaxChunkIndices-2:{syntaxChunkIndices}")
                if len(syntaxChunkIndices) != len(syntaxChunk.tokens):
                    # 3. If the above mechanism doesn't work, we use all the phrase tokens as alignment base 
                    syntaxChunkIndices = tokenSubset[tokenSubset.str.lower().isin(syntaxChunk.tokens)].index.tolist()
                    print(f"syntaxChunkIndices-3:{syntaxChunkIndices}")
                    if len(syntaxChunkIndices) != len(syntaxChunk.tokens):
                        if len(syntaxChunk.tokens) == 1:
                            # 4. If only one token in the syntaxChunk and multi indices are found, we keep the closest index to the 
                            # last index in the list
                            for index in syntaxChunkIndices:
                                if index > syntaxChunkIdxGroups_withData[-1]['indices'][-1]:
                                    syntaxChunkIndices = [index]
                                    break
                        else:
                            # 5. We remove additonal aligned tokens by checking the token number and the token interval
                            syntaxChunkIndices = trimIndices(syntaxChunkIndices,len(syntaxChunk.tokens))
                            print(f"syntaxChunkIndices-4:{syntaxChunkIndices}")
            assert len(syntaxChunkIndices) == len(syntaxChunk.tokens)
            syntaxChunkIdxGroups_withData.append({
                "indices": syntaxChunkIndices,
                "extra_str": f"{syntaxChunk.syntaxType}|{syntaxChunk.partOfSpeech}"
            })
            syntaxTokenIdxOffset = syntaxTokenIdxOffset_next
        for conceptGroup in phrase.mappings:
            for concept in conceptGroup.concepts:
                conceptIdxList_flatten = resolveTokenIndices_byPosition_multiToken(tokenOffset, concept.startPosList, concept.lengthList)
                conceptIdxGroup_withData.append({
                    "indices":conceptIdxList_flatten,
                    "extra_str":f"{conceptGroupId}|{concept.umlsCUI}|{concept.preferedName}({concept.hitTerm})|{','.join(concept.categories)}|{concept.isHead}|{concept.isNegated}",
                })
            conceptGroupId += 1
    for negation in metamapRecord.getFindingSection().negations:
        negTriggerIdxList_flatten = resolveTokenIndices_byPosition_multiToken(tokenOffset, negation.trgger['startPosList'], negation.trgger['lengthList'])
        negTargetIdxList_flatten = resolveTokenIndices_byPosition_multiToken(tokenOffset, negation.tarrget['startPosList'], negation.tarrget['lengthList'])
        negTriggerGroups_withData.append({
            "indices":negTriggerIdxList_flatten,
            "extra_str":f"{negationGroupId}|{','.join([str(i) for i in negTargetIdxList_flatten])}|{','.join(negation.tarrget['conceptsCUIs'])}"
        })
        negTargetGroups_withData.append({
            "indices":negTargetIdxList_flatten,
            "extra_str":f"{negationGroupId}|{','.join([str(i) for i in negTriggerIdxList_flatten])}|{','.join(negation.tarrget['conceptsCUIs'])}"
        })
        negationGroupId += 1
    return phraseIdxGroups, syntaxChunkIdxGroups_withData, conceptIdxGroup_withData, negTriggerGroups_withData, negTargetGroups_withData

Run

In [117]:
SPACY_PREFIXX = "[sp]"
SPACY_COLUMN_NAME = {
    'token': SPACY_PREFIXX+'token',
    'token_offset':SPACY_PREFIXX+'token_offset',
    'sentence_group':SPACY_PREFIXX+'sentence_group',
    'noun_chunk':SPACY_PREFIXX+'noun_chunk',
    'lemma':SPACY_PREFIXX+'lemma',
    'pos_core':SPACY_PREFIXX+'pos_core',
    'pos_feature':SPACY_PREFIXX+'pos_feature',
    'dependency':SPACY_PREFIXX+'dependency',
    'dependency_head':SPACY_PREFIXX+'dependency_head',
    'dependency_children':SPACY_PREFIXX+'dependency_children',
    'morphology':SPACY_PREFIXX+'morphology',
    'is_alpha':SPACY_PREFIXX+'is_alpha',
    'is_stop':SPACY_PREFIXX+'is_stop',
    'is_pronoun':SPACY_PREFIXX+'is_pronoun',
    'trailing_space':SPACY_PREFIXX+'trailing_space',
}
METAMAP_PREFIXX = "[mm]"
METAMAP_COLUMN_NAME = {
    'phrase': METAMAP_PREFIXX+'metamap_phrase',
    'syntax_chunk': METAMAP_PREFIXX+'syntax_chunk|syntax_type|pos',
    'concept': METAMAP_PREFIXX+'concept_group|CUI|prefered_name(hit_synonym)|categories|isHead|isNegated',
    'neg_trigger': METAMAP_PREFIXX+'negation_group|target_token_indices|target_CUI',
    'negated_target': METAMAP_PREFIXX+'negation_group|trigger_token_indices|target_CUI'
}

In [118]:
import stanza
from stanza.server import CoreNLPClient

In [158]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Lock
from IPython.display import display, HTML
import json
import spacy
import stanza
from stanza.server import CoreNLPClient


BATCH_SIZE = 10
# mp.cpu_count()
WORKERS_NUM = 8
DATA_START_POS = 0 #8690
DATA_END_POS = 100
# DATA_END_INDEX = DATA_SIZE

CORENLP_CUSTOM_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, depparse, coref',
    "coref.algorithm": "statistical"
}

nlp = spacy.load("en_core_web_md", disable=['ner'])

executor = ProcessPoolExecutor(max_workers=WORKERS_NUM)
all_task = [executor.submit(run_metamap, startIndex, BATCH_SIZE) for startIndex in range(DATA_START_POS, DATA_END_POS, BATCH_SIZE)]

lock=Lock()
with open("/home/yuxiangliao/PhD/output/metamap/metamap_output_test.json","w") as f:
    # CoreNLPClient(memory='8G', threads=8, endpoint='http://localhost:8801', 
    #               be_quiet=True, properties=CORENLP_CUSTOM_PROPS) as client:
    for future in as_completed(all_task):
        # Metamap
        metamap_output, idx_inteval = future.result()
        id_subList = id_list[idx_inteval[0]:idx_inteval[1]] 
        metamap_json_output = list(metamap_output.split("\n"))[1] # Only the second line is the required JSON string.
        metamap_json_obj = json.loads(metamap_json_output)
        parsed_obj_batch = parseMetamapJSON(metamap_json_obj, id_subList)
        # print(classToJSON(parsed_obj_batch))
    
        # SpaCy
        text_tuples = [(record.getFindingSection().text,{"record":record}) for record in parsed_obj_batch.records]
        for doc, context in nlp.pipe(text_tuples, as_tuples=True):
            data = {
                SPACY_COLUMN_NAME['token']: [tok.text for tok in doc],
                SPACY_COLUMN_NAME['token_offset']: getTokenOffset(doc.text, doc),
                SPACY_COLUMN_NAME['sentence_group']: align(doc, doc.sents),
                SPACY_COLUMN_NAME['noun_chunk']: align(doc, doc.noun_chunks),
                SPACY_COLUMN_NAME['lemma']: [tok.lemma_ for tok in doc],
                SPACY_COLUMN_NAME['pos_core']: [f"[{tok.pos_}]{spacy.explain(tok.pos_)}" for tok in doc],
                SPACY_COLUMN_NAME['pos_feature']: [f"[{tok.tag_}]{spacy.explain(tok.tag_)}" for tok in doc],
                SPACY_COLUMN_NAME['dependency']: [f"[{tok.dep_}]{spacy.explain(tok.dep_)}" for tok in doc],
                SPACY_COLUMN_NAME['dependency_head']: [tok.head.text for tok in doc],
                SPACY_COLUMN_NAME['dependency_children']: [[child for child in tok.children] for tok in doc],
                SPACY_COLUMN_NAME['morphology']: [tok.morph for tok in doc],
                SPACY_COLUMN_NAME['is_alpha']: [tok.is_alpha for tok in doc],
                SPACY_COLUMN_NAME['is_stop']: [tok.is_stop for tok in doc],
                SPACY_COLUMN_NAME['is_pronoun']: [True if tok.pos_ == 'PRON' else False for tok in doc],
                SPACY_COLUMN_NAME['trailing_space']: [True if tok.whitespace_ else False for tok in doc],
            }
            spacyOutput = pd.DataFrame(data=data)
            
            # Metamap
            metamapRecord = context['record'] # the Record obj resolved from metamap output
            print(metamapRecord.sid)
            # display(HTML(spacyOutput.to_html()))
            phraseInfo, syntaxChunkInfo, conceptInfo, negTriggerInfo, negTargetInfo = formatMetamapRecord(metamapRecord)
                
            metamapOutput = pd.DataFrame({
                METAMAP_COLUMN_NAME['phrase']: align_byIndex(doc, phraseInfo),
                METAMAP_COLUMN_NAME['syntax_chunk']: align_byIndex_individually_withData_noOverlap(doc, syntaxChunkInfo),
                METAMAP_COLUMN_NAME['concept']: align_byIndex_individually_withData(doc,conceptInfo),
                METAMAP_COLUMN_NAME['neg_trigger']: align_byIndex_individually_withData(doc,negTriggerInfo),
                METAMAP_COLUMN_NAME['negated_target']: align_byIndex_individually_withData(doc,negTargetInfo),
            })
            
            output = spacyOutput.join(metamapOutput)
            
            
            # CoreNLP
            # document = client.annotate(metamapRecord.getFindingSection().text)

### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.
### WARNING: Overriding default model 2020AA with 2022AA.


s58350684
tokenSubset:['The', 'cardiac', 'silhouette']
syntaxChunk.text:The
syntaxChunk.tokens:['the']
syntaxChunkIndices-1:[0]
syntaxChunk.text:cardiac
syntaxChunk.tokens:['cardiac']
syntaxChunkIndices-1:[1]
syntaxChunk.text:silhouette
syntaxChunk.tokens:['silhouette']
syntaxChunkIndices-1:[2]
tokenSubset:['is']
syntaxChunk.text:is
syntaxChunk.tokens:['is']
syntaxChunkIndices-1:[3]
tokenSubset:['normal', 'in', 'size', '.']
syntaxChunk.text:normal
syntaxChunk.tokens:['normal']
syntaxChunkIndices-1:[4]
syntaxChunk.text:in
syntaxChunk.tokens:['in']
syntaxChunkIndices-1:[5]
syntaxChunk.text:size
syntaxChunk.tokens:['size']
syntaxChunkIndices-1:[6]
syntaxChunk.text:.
syntaxChunk.tokens:['.']
syntaxChunkIndices-1:[7]
tokenSubset:['The', 'hilar']
syntaxChunk.text:The
syntaxChunk.tokens:['the']
syntaxChunkIndices-1:[9]
syntaxChunk.text:hilar
syntaxChunk.tokens:['hilar']
syntaxChunkIndices-1:[10]
tokenSubset:['and']
syntaxChunk.text:and
syntaxChunk.tokens:['and']
syntaxChunkIndices-1:[11]
toke

### WARNING: Overriding default model 2020AA with 2022AA.


s54828594
tokenSubset:['Mild', 'to', 'moderate', 'enlargement', 'of', 'the', 'cardiac', 'silhouette']
syntaxChunk.text:Mild to moderate
syntaxChunk.tokens:['mild', 'to', 'moderate']
syntaxChunkIndices-1:[0, 1, 2]
syntaxChunk.text:enlargement
syntaxChunk.tokens:['enlargement']
syntaxChunkIndices-1:[3]
syntaxChunk.text:of
syntaxChunk.tokens:['of']
syntaxChunkIndices-1:[4]
syntaxChunk.text:the
syntaxChunk.tokens:['the']
syntaxChunkIndices-1:[5]
syntaxChunk.text:cardiac
syntaxChunk.tokens:['cardiac']
syntaxChunkIndices-1:[6]
syntaxChunk.text:silhouette
syntaxChunk.tokens:['silhouette']
syntaxChunkIndices-1:[7]
tokenSubset:['is']
syntaxChunk.text:is
syntaxChunk.tokens:['is']
syntaxChunkIndices-1:[8]
tokenSubset:['unchanged', '.']
syntaxChunk.text:unchanged
syntaxChunk.tokens:['unchanged']
syntaxChunkIndices-1:[9]
syntaxChunk.text:.
syntaxChunk.tokens:['.']
syntaxChunkIndices-1:[10]
tokenSubset:['The', 'aorta']
syntaxChunk.text:The
syntaxChunk.tokens:['the']
syntaxChunkIndices-1:[12]
syntaxC

### WARNING: Overriding default model 2020AA with 2022AA.


s51181158
tokenSubset:['Portable', 'AP', 'chest', 'radiograph', '.']
syntaxChunk.text:Portable
syntaxChunk.tokens:['portable']
syntaxChunkIndices-1:[0]
syntaxChunk.text:AP
syntaxChunk.tokens:['ap']
syntaxChunkIndices-1:[1]
syntaxChunk.text:chest radiograph
syntaxChunk.tokens:['chest', 'radiograph']
syntaxChunkIndices-1:[2, 3]
syntaxChunk.text:.
syntaxChunk.tokens:['.']
syntaxChunkIndices-1:[4]
tokenSubset:['The', 'lungs']
syntaxChunk.text:The
syntaxChunk.tokens:['the']
syntaxChunkIndices-1:[6]
syntaxChunk.text:lungs
syntaxChunk.tokens:['lungs']
syntaxChunkIndices-1:[7]
tokenSubset:['are']
syntaxChunk.text:are
syntaxChunk.tokens:['are']
syntaxChunkIndices-1:[8]
tokenSubset:['relatively']
syntaxChunk.text:relatively
syntaxChunk.tokens:['relatively']
syntaxChunkIndices-1:[9]
tokenSubset:['well']
syntaxChunk.text:well
syntaxChunk.tokens:['well']
syntaxChunkIndices-1:[10]
tokenSubset:['expanded', 'without', 'focal', 'consolidation', ',']
syntaxChunk.text:expanded
syntaxChunk.tokens:['expand

AssertionError: 

Check metamap output

In [ ]:
output[output[SPACY_COLUMN_NAME['pos_core']].str.contains("PRON")]

In [ ]:
output.to_csv("/home/yuxiangliao/PhD/output/s53741303_test.csv")

In [152]:
from IPython.display import display, HTML
display(HTML(output.to_html()))

,[sp]token,[sp]token_offset,[sp]sentence_group,[sp]noun_chunk,[sp]lemma,[sp]pos_core,[sp]pos_feature,[sp]dependency,[sp]dependency_head,[sp]dependency_children,[sp]morphology,[sp]is_alpha,[sp]is_stop,[sp]is_pronoun,[sp]trailing_space,[mm]metamap_phrase,[mm]syntax_chunk|syntax_type|pos,[mm]concept_group|CUI|prefered_name(hit_synonym)|categories|isHead|isNegated,[mm]negation_group|target_token_indices|target_CUI,[mm]negation_group|trigger_token_indices|target_CUI
0,The,0,0,0,the,[DET]determiner,[DT]determiner,[det]determiner,cardiac,[],"(Definite=Def, PronType=Art)",True,True,False,True,0,0|det|det,-1,-1,-1
1,cardiac,4,0,0,cardiac,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (Chinese)",[amod]adjectival modifier,contours,"[The, ,]",(Degree=Pos),True,False,False,False,0,1|mod|adj,"[0|C0018787|Heart(CARDIAC)|bpoc|0|0, 1|C0018787|Heart(CARDIAC)|bpoc|0|0, 2|C1314974|Cardiac attachment(Cardiac)|inpr|0|0, 3|C1314974|Cardiac attachment(Cardiac)|inpr|0|0, 4|C1522601|Cardiac - anatomy qualifier(Cardiac)|qlco|0|0, 5|C1522601|Cardiac - anatomy qualifier(Cardiac)|qlco|0|0]",-1,-1
2,",",11,0,0,",",[PUNCT]punctuation,"[,]punctuation mark, comma",[punct]punctuation,cardiac,[],(PunctType=Comm),False,False,False,True,0,2|punc|,-1,-1,-1
3,mediastinal,13,0,0,mediastinal,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (Chinese)",[amod]adjectival modifier,contours,"[and, hilar]",(Degree=Pos),True,False,False,True,0,3|head|adj,"[0|C0025066|Mediastinum(Mediastinal)|blor|1|0, 1|C1522718|Mediastinal(Mediastinal)|spco|1|0, 2|C0025066|Mediastinum(Mediastinal)|blor|1|0, 3|C1522718|Mediastinal(Mediastinal)|spco|1|0, 4|C0025066|Mediastinum(Mediastinal)|blor|1|0, 5|C1522718|Mediastinal(Mediastinal)|spco|1|0]",-1,-1
4,and,25,0,0,and,[CCONJ]coordinating conjunction,"[CC]conjunction, coordinating",[cc]coordinating conjunction,mediastinal,[],(ConjType=Cmp),True,True,False,True,1,4|conj|conj,-1,-1,-1
5,hilar,29,0,0,hilar,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (Chinese)",[conj]conjunct,mediastinal,[],(Degree=Pos),True,False,False,True,2,5|mod|adj,[6|C0205150|Hilar(HILAR)|spco|0|0],-1,-1
6,contours,35,0,0,contour,[NOUN]noun,"[NNS]noun, plural",[nsubj]nominal subject,are,"[cardiac, mediastinal]",(Number=Plur),True,False,False,True,2,6|head|noun,[6|C0876954|Contour form(Contours)|spco|1|0],-1,-1
7,are,44,0,-1,be,[AUX]auxiliary,"[VBP]verb, non-3rd person singular present",[ROOT]root,are,"[contours, normal, .]","(Mood=Ind, Tense=Pres, VerbForm=Fin)",True,True,False,True,3,7|aux|aux,-1,-1,-1
8,normal,48,0,-1,normal,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (Chinese)",[acomp]adjectival complement,are,[],(Degree=Pos),True,False,False,False,4,8|head|adj,"[7|C0040300|Body tissue(Normal)|tisu|1|0, 8|C0205307|Normal(NORMAL)|qlco|1|0, 9|C0232357|Normal Capillary Refill Time(Normal)|fndg|1|0, 10|C0439166|Percent normal(% normal)|qnco|1|0, 11|C2347086|Mean Percent of Normal(% Normal)|qnco|1|0, 12|C4553972|How Often Felt Normal question(Normal)|inpr|1|0, 13|C5200925|Normal Gait(Normal)|fndg|1|0, 14|C5202633|Normal Immune Presence(Normal)|fndg|1|0, 15|C5235018|Normal Abdomen on Palpation(Normal)|fndg|1|0, 16|C5235188|Normal Precordial Palpation Finding(Normal)|fndg|1|0, 17|C5235189|Normal Abdomen on Visual Inspection(Normal)|fndg|1|0, 18|C5235193|Normal Chest Appearance(Normal)|fndg|1|0, 19|C5235194|Normal Appearance of Extremities(Normal)|fndg|1|0, 20|C5235195|Normal Male External Genitalia(Normal)|fndg|1|0, 21|C5238822|Normal Point of Maximum Impulse(Normal)|fndg|1|0]",-1,-1
9,.,54,0,-1,.,[PUNCT]punctuation,"[.]punctuation mark, sentence closer",[punct]punctuation,are,[],(PunctType=Peri),False,False,False,True,4,9|punc|,-1,-1,-1
